# Exemplo do OpenMP
#### _bootcamp_ da Escola Supercomputador Santos Dumont - 2025
por Calebe de Paula Bianchini

### Você lembra do Método do Trapézio?

![Método do Trapézio](./img/mpi_trapezio.png?raw=1)

O método do trapézio estima o valor de f(x)
dividindo o intervalo ${[a; b]}$ em $n$ segmentos iguais e calculando a
seguinte equação:

$$h * [\frac{f(x_o)}{2} + \frac{f(x_n)}{2} + \displaystyle \sum_{i=1}^{n-1} f(x_i)]$$

Onde:

$h =$ $\frac{(b-a)}{n}$ e $x_i=a + i*h, i = 1,...,(n-1)$

E, considerando $f(x)$ como uma função já predefinida, o código
a seguir é uma possível versão sequencial para esse cálculo:

In [1]:
%%writefile trapezio.c

#include <stdio.h>
#include <math.h>

/* Calcula f(x). */
double f(double x)
{
  double return_val;
  return_val = exp(x);
  return return_val;
}

int main(int argc, char *argv[])
{
  double integral; /* integral armazena resultado final */
  double a, b;     /* a, b -  limite esquerdo e direito da função */
  long i, n;       /* n - número de trapezóides */
  double x, h;     /* h - largura da base do trapezóide */

  a = 0.0;
  b = 1.0;
  n = 800000000; // -> este e um numero bem grande: 8000000000;
  // retire o comentário abaixo para fazer a leitura do valor
  // printf("Entre a, b, e n \n");
  // scanf("%f %f %d", &a, &b, &n);
  h = (b - a) / n;
  integral = (f(a) + f(b)) / 2.0;
  x = a;
  for (i = 1; i < n; i++) {
    x += h;
    integral += f(x);
  }
  integral *= h;
  printf("Com n = %ld trapezóides, a estimativa \n", n);
  printf("da integral de %lf até %lf = %lf\n", a, b, integral);
  return (0);
}

Writing trapezio.c


Em seguida, compilamos o nosso código:

In [2]:
!gcc -fopenmp trapezio.c -o trapezio -Wall -lm -O3

Por fim, vamos executar e descobrir quantas _threads_ foram criadas.

In [3]:
!time -p ./trapezio

Com n = 800000000 trapezóides, a estimativa 
da integral de 0.000000 até 1.000000 = 1.718282
real 18.36
user 14.80
sys 0.11


Já podemos perceber, nessa execução, que o tempo gasto na computação desse problema usando 1 CPU é bem alto (quanto maior o valor de _n_, melhor a precisão do resultado, mas muito mais será a compiutação). O que queremos, a partir desse ponto, é utilizar todos os recursos disponíveis no computador para calcular o resultado final.

Felizmente, o método do trapézio tem grande potencial de paralelismo. Cada iteração do laço de repetição _for_ pode ser executado de forma independente. Por isso, podemos usar a diretiva __#pragma omp for__. Com isso, cada iteraçào será executada em uma _thread_.

Além disso, conhecendo o _tempo_ de computação em cada iteração do laço e sendo eles praticamente iguais, foi feita a escolha de particionamento das iterações pelas _threads_ de forma estática: _static_. Ou seja, a divisão e a atribição de cada iteração para as _threads_ disponíveis no ambiente são feitas antes do início da execução do laço. Portanto, teremos _n/p_ iterações para cada _thread_, onde _p_ é a quantidade de _threads_ disponíveis na região paralela.

Por fim, sabendo que o resultado de cada iteração será _reduzido_ em uma única variável final (a famosa variável _acumuladora_), vamos utilizar a cláusula _reduction_ para que todo o controle de sinconização (seção crítica) do acesso à variável seja delegado para o OpenMP.

O resutlado desse novo código pode ser visto a seguir:

In [4]:
%%writefile omp_trapezio.c

#include <stdio.h>
#include <math.h>
#include <omp.h>

double f(double x)
{
  double valor;
  valor = exp(x);
  return (valor);
}

int main(int argc, char *argv[])
{
  double integral; /* Armazena resultado em integral */
  double a, b;     /* Limite esquerdo e direito */
  long n;          /* Número de Trapezóides */
  double h;        /* Largura da base do Trapezóide */
  a = 0.0;
  b = 1.0;
  n = 800000000; // -> este e um numero bem grande: 8000000000;
  // retire o comentário abaixo para fazer a leitura do valor
  // printf("Entre a, b, e n \n");
  // scanf("%lf %lf %ld", &a, &b, &n);
  h = (b - a) / n;
  integral = (f(a) + f(b)) / 2.0;

#pragma omp parallel for default(none) firstprivate(n, a, h) reduction(+ : integral) schedule(static)
  for (long i = 1; i < n - 1; i++) {
    integral += f(a + i * h);
  }
  integral *= h;

  printf("Com n = %ld trapezoides, a estimativa \n", n);
  printf("da integral de %lf ate %lf = %lf \n", a, b, integral);
  printf("%d threads.\n", omp_get_max_threads());
  return (0);
}

Writing omp_trapezio.c


Em seguida compilamos e executamos o programa (com a medição de tempo, novamente):

In [5]:
!gcc -fopenmp omp_trapezio.c -o omp_trapezio -Wall -lm -O3
!time -p ./omp_trapezio

Com n = 800000000 trapezoides, a estimativa 
da integral de 0.000000 ate 1.000000 = 1.718282 
2 threads.
real 10.92
user 17.73
sys 0.04


A partir dessa execução, é possível comparar a execução sequencial com a execução paralela para saber, ao menos, o quão mais rápida foi essa nova solução.

Por isso, podemos executar diversas vezes cada uma das soluções para que, estatisticamente, tenhamos um conjunto de medições de tempo aceitável. A média dessas execuções pode ser utilizada para avaliar o desempenho da solução.

Além disso, ao variar a quantidade de _threads_ (preferencialmente em um equipamento com _muitos cores_), será possível averiguar se aquela quantidade aumentar o desempenho conforme aumentamos a quantidade de _cores_.

Com esses números é possível calcular uma métrica chamada de _SpeedUp_. Ela mede a razão entre o tempo gasto para
execução de um algoritmo ou aplicação em um único processador ($T(1)$) e
o tempo gasto na execução com $P$ processadores ($T(P)$), como visto na
equação a seguir:

$$S(P) = \frac{T(1)} {T(P)}$$

Em condições ideais, quando o *speedup* é sempre igual a $P$, onde $P$ é
o número de processadores em uso, temos o chamado *speedup* linear. Mas,
em geral, o *speedup* é menor do que $P$, devido principalmente à
sobrecarga de comunicação entre os diferentes fluxos de execução do
programa, perdas por sincronização e decomposição de tarefas mal feita.
Quando isso acontece, chamamos o *speedup* de sublinear. Essa situação
pode se deteriorar até o ponto no qual a adição de mais processadores
diminui o ganho obtido, em vez de aumentar, caracterizando um "retorno
negativo".
Estas e outras situações podem ser vistas no exemplo de cruso de _speedup_ a seguir:

![Curvas de speedup](./img/teo_speedup.png)

No caso do problema do trapézio trapézio, a análise de desempenho registrada na literatura (ver o livro referenciado a seguir) permite confirmar que seu comportamento segue um *speedup* linear ideal. Esse comportamento é esperado para o problema do
trapézio, pois o cálculo de cada iteração pode ser realizado de forma
independente. Veja o gráfico a seguir.

Apesar disso, sabemos que essa solução possui uma fração
não paralelizável e, como consequência, esse trecho afeta a
escalabilidade.

![Speedup Trapézio](./img/omp_trapezio_speedup.png)

Maiores detalhes sobre Programação Paralela e OpenMP, vejam:

* __Programação Paralela e Distribuída__ _com MPI, OpenMP e OpenACC para computação de alto desempenho_, em [Casa do Código](https://www.casadocodigo.com.br/products/livro-programacao-paralela).

